# Introduction to CGAP data objects #

CGAP data universe begins with the original 18 CGAP surveys (three surveys for each of six countries).  The data are cleaned and aligned, and secondary data products are built, as shown in the following data map.  Most of these data products are flat files that lack metadata.  This notebook describes my first attempt at a json-based representation of data objects.  

In [1]:
import sys
import os, json
import copy
from types import SimpleNamespace

import numpy as np
import pandas as pd

# Change this filepath to one for your machine
sys.path.append('/Users/mordor/research/habitus_project/mycode/predictables/Code and Notebooks')

from CGAP_JSON_Encoders_Decoders import Question_Decoder, CGAP_Encoded, CGAP_Decoded, Country_Decoded


# Change this filepath to one for your machine. The actual file is on our Box
# folder at https://pitt.app.box.com/folder/136317983622

Data = CGAP_Decoded()
Data.read_and_decode('/Users/mordor/research/habitus_project/mycode/predictables/Data/Data Objects/CGAP_JSON.txt')

countries = ['bgd','cdi','moz','nga','tan','uga']

ModuleNotFoundError: No module named 'numpy'

Before you begin, go to https://pitt.app.box.com/folder/134211910534 and open one of the User Guides.  Scroll down to page 30 or thereabouts -- it varies by country -- and familiarize yourself with the three surveys (Household, Multiple-respondent and Single-respondent).  Note that each survey item has an label such as H28 or A13. These are unique: No label appears in more than one survey.  In general, though not without exception, the same question in different countries gets the same label.

The `Data` object holds decoded json strings that hold data and metadata for each question.  `Data` is actually a Python SimpleNamespace, which makes it easy to access data using dot notation.   Thus, `Data.__dict__.keys()` gives you all the question keys:

In [2]:
Data.__dict__.keys()

dict_keys(['bgd_A1', 'cdi_A1', 'moz_A1', 'nga_A1', 'tan_A1', 'uga_A1', 'bgd_A2', 'cdi_A2', 'moz_A2', 'nga_A2', 'tan_A2', 'uga_A2', 'bgd_A3', 'cdi_A3', 'moz_A3', 'nga_A3', 'tan_A3', 'uga_A3', 'bgd_A4', 'cdi_A4', 'moz_A4', 'nga_A4', 'tan_A4', 'uga_A4', 'bgd_A5', 'cdi_A5', 'moz_A5', 'nga_A5', 'tan_A5', 'uga_A5', 'bgd_A7', 'cdi_A7', 'moz_A7', 'nga_A7', 'tan_A7', 'uga_A7', 'bgd_A9', 'cdi_A9', 'moz_A9', 'nga_A9', 'tan_A9', 'uga_A9', 'bgd_A25', 'cdi_A25', 'moz_A25', 'nga_A25', 'tan_A25', 'uga_A25', 'bgd_A36', 'cdi_A36', 'moz_A36', 'nga_A36', 'tan_A36', 'uga_A36', 'bgd_A53', 'cdi_A53', 'moz_A53', 'nga_A53', 'tan_A53', 'uga_A53', 'bgd_A6', 'cdi_A6', 'moz_A6', 'nga_A6', 'tan_A6', 'uga_A6', 'bgd_A8', 'cdi_A8', 'moz_A8', 'nga_A8', 'tan_A8', 'uga_A8', 'bgd_A26', 'cdi_A26', 'moz_A26', 'nga_A26', 'tan_A26', 'uga_A26', 'bgd_A37', 'cdi_A37', 'moz_A37', 'nga_A37', 'tan_A37', 'uga_A37', 'bgd_A10', 'cdi_A10', 'moz_A10', 'nga_A10', 'tan_A10', 'uga_A10', 'bgd_A11', 'cdi_A11', 'moz_A11', 'nga_A11', 'tan_A11'

The syntax of data object identifiers is simply `country+'_'+label`. If you ask for, say, `Data.uga_A1`, you get all the metadata for question `A1` and also the data -- the answers in the Uganda survey to question `A1`:

In [3]:
Data.uga_A1

namespace(qtype='single',
          text='What is the form of ownership of your land?',
          label='A1',
          answers={'lease_certificate': 1,
                   'customary_law': 2,
                   'communal': 3,
                   'state_ownership': 4,
                   'other': 5},
          country='uga',
          survey='rr',
          df_name='uga_rr',
          df=           A1
             23068679  2.0
             22716424  1.0
             22716425  3.0
             23306249  1.0
             22945803  1.0
             ...       ...
             23306221  2.0
             23306222  2.0
             22872056  2.0
             22904826  1.0
             22945787  1.0
             
             [2859 rows x 1 columns])

Question A1 asks "What is the form of ownership of your land?".  It is a single-answer question (`qtype = 'single'`) which means it expects a single answer that is coded as an integer between 1 and 5. The mapping from answers to integers is described in the `answers` dict. 

Look at question A1 in any User Guide and you'll see that the text strings in the `answers` dict are not identical with the actual answers in the User Guide, but are short mnemonics.

The other metadata fields are 

- `label`: the unique identifier of a question
- `country` : an identifier of the country from which the data is sourced
- `survey` : refers to intermediate data products (see data map, above).  
- `df` : a pandas dataframe that holds the data, itself.

We'll look at the `df` dataframes after I introduce single- and multiple-answer questions. 


### CGAP_Decoded API ###

CGAP_Decoded objects have several methods for viewing data and meta-data and assembling dataframes from data objects.  For a CGAP_Decoded object called Data, these methods are:

- `decode(jstring)` : decodes a JSON string representation of data object(s)
- `read_and_decode(file)` : reads JSON from file and decodes it
- `by_name(name)` : convenience function to get a data object by its name; equivalent to Data.__dict__.get(name)
- `add_col(country, label, df, qtype = 'single', text = None, answers = None)` : adds a column to Data. This is used to add derived variables (e.g., a log transform of a variable, or cluster labels, etc.) See section on `add_col`, below. 
- `describe(label, country = 'bgd', display = True)` : prints a short text description of a variable given its label.  If display=False, this returns an f-string representation of the description.
- `col(country,label,*column)`: the workhorse method for assembling pandas dataframes from data objects. Given a country and a label, this returns a dataframe. The optional `*column` argument is for multi-answer questions; see below.
- `col_from_countries(var, countries)` : This assembles a dataframe for one variable from all the specified countries. var is either a label, alone, if the label denotes a single-answer question, or a (label, column_name) tuple if the label denotes a multi-answer question. For example, `col_from_countries(('A5','Rice'),['bgd','nga'])` assembles a dataframe of the `Rice` answer to question `A5` for Bangladesh and Nigeria.
- `cols_from_countries(*vars,countries)`: Assembles a dataframe for all the variables for all the countries. 


Most of these methods are described in some detail later in this Introduction.

### Countries don't always have the same answers to questions ###

One tricky aspect of the CGAP data is that a given question won't always have the same answers in different countries.  Here's a description of `A1` for Uganda:

In [4]:
Data.describe('A1','uga')

label : A1
 text : What is the form of ownership of your land?
 qtype : single
 survey : rr
 answers : {'lease_certificate': 1, 'customary_law': 2, 'communal': 3, 'state_ownership': 4, 'other': 5}



Bangladesh has an additional answer -- Kott -- which it codes as 5:

In [5]:
Data.describe('A1','bgd')

label : A1
 text : What is the form of ownership of your land?
 qtype : single
 survey : rr
 answers : {'lease_certificate': 1, 'customary_law': 2, 'communal': 3, 'state_ownership': 4, 'Kott': 5, 'other': 6}



So 5 means 'other' in Uganda's survey and 'Kott' in Bangladesh's.   Sometimes this variability in coding schemes is so extreme that it's worth recoding the individual country data to a inter-country scheme (crops and livestock are a good example, see below), but in most cases, like question `A1`, I have not tried to recode all answers to an inter-country scheme, so you should be attentive to the `answers` dict and the mapping of answers to numeric codes.

Sometimes, answers aren't recoded but simply recorded "as is", in which case there won't be an `answers` dict.  For example, question `A2` asks how much land is owned:

In [6]:
Data.describe('A2','bgd')

label : A2
 text : How many hectares of agricultural land do you own?
 qtype : single
 survey : rr
 answers : None



## Single- and multiple-answer questions ##

Attributes such as farm size or family size have just one answer per household, but attributes such as the crops grown by a household can have several values. There's no easy way to code multiple answers to one question in a single variable, so CGAP uses multiple variables to code the answers to multi-answer questions. We've seen examples of single-answer questions above, now let's look at multi-answer questions, such as `H17`, which asks how important it is to save money for various purposes:

In [7]:
Data.moz_H17

namespace(qtype='multi',
          text=' In your opinion, how important is it for your household to save for each of the following?',
          label='H17',
          answers={'very_important': 1,
                   'somewhat_important': 2,
                   'not_important': 3},
          column_dict={'future_purchases': 1,
                       'unexpected_event': 2,
                       'regular_purchases': 3,
                       'school_fees': 4},
          country='moz',
          survey='sr',
          df_name='moz_sr',
          df=          future_purchases  unexpected_event  regular_purchases  school_fees
             21948170               1.0               1.0                1.0          1.0
             21948180               2.0               1.0                2.0          1.0
             21948779               1.0               1.0                1.0          1.0
             21951744               1.0               1.0                2.0          1.0
           

The Data object for `H17` for Mozambique has a field called `column_dict` that you won't see in single-answer Data objects, and the `df` field contains a multi-column pandas dataframe. Each of these columns contains numbers that encode the answers described in the `answers` dict; that is, 1, 2 or 3 depending on whether the answer is "very_important", "somewhat important" or "not important", respectively. 

Look at the first row of the dataframe:  It tells us that the survey respondent from the household with ID 21948170 thinks it is very important to save for all four kinds of expenses; whereas the second household thinls it is only somewhat important to save for future purchases and regular purchases. 

The dataframe for a multi-answer question has one column per answer, and the names of the columns are the mnemonic strings in `column_dict`.  Thus, you can easily find out which farmers in Uganda grow tomatoes:

In [8]:
Data.moz_A5.df.Tomatoes

22552580    1.0
22487045    2.0
22159366    2.0
22790149    1.0
22790150    2.0
           ... 
22757331    2.0
22552539    2.0
22200293    2.0
22102008    2.0
22167545    2.0
Name: Tomatoes, Length: 2462, dtype: float64

Let me break down this query: 

- `Data` is a namespace 
- `moz_A5` is a key into `Data.__dict__` that returns an object that represents question `A5` for Mozambique
- `df` is the field of the object that contains a pandas dataframe
- `Tomatoes` is a column in this dataframe

The df columns for multi-answer questions are always mnemonics for answers.  This is for two reasons:  It's easier for the user to ask for `df.Tomatoes` than, say, `df.A5_27`. More importantly, while `df.A5_27` contains data about tomato-growing in Mozambique, it contains data about growing sesame in Bangladesh and sugarcane in Tanzania.  These differences in coding have all been resolved behind the scenes.

You can get the same data using the `col` method, described below.  

In [9]:
Data.col('moz','A5','Tomatoes')

22552580    1.0
22487045    2.0
22159366    2.0
22790149    1.0
22790150    2.0
           ... 
22757331    2.0
22552539    2.0
22200293    2.0
22102008    2.0
22167545    2.0
Name: Tomatoes, Length: 2462, dtype: float64

## Household IDs and DataFrame Joins ##

You'll notice that the dataframes associated with Data objects have household ids as their indices.  For example, the first record in any Mozambique Data object can be obtained in the usual pandas way:

In [10]:
Data.moz_A5.df.loc[22552580]

Maize           2.0
Beans           2.0
Sweet_potato    2.0
Sorghum         2.0
Rice            2.0
Groundnuts      1.0
Cowpea          2.0
Millet          2.0
Cassava         1.0
Potato          2.0
Pigeon_pea      2.0
Banana          2.0
Coconut         2.0
Cotton          2.0
Sesame          2.0
Mango           2.0
Cashew          2.0
Sugarcane       2.0
Tobacco         2.0
Tea             2.0
Avocado         2.0
Cocoa           2.0
Sisal           2.0
Cloves          2.0
Coffee          2.0
Sunflower       2.0
Tomatoes        1.0
Onions          2.0
Other_1         2.0
Other_2         2.0
Other_3         2.0
No_crop         1.0
Name: 22552580, dtype: float64

This is answers to question `A5` for the household with `HHID` 22552580.  Similarly, here is the answer to the earlier question about farm size:

In [11]:
Data.moz_A2.df.loc[22552580]

A2    5.0
Name: 22552580, dtype: float64

Because Data object `df` indices are unique household IDs, it is straightforward to join `df`s:

In [12]:
x = Data.moz_A2.df.join(Data.moz_A5.df)
x

,A2,Maize,Beans,Sweet_potato,Sorghum,Rice,Groundnuts,Cowpea,Millet,Cassava,...,Sisal,Cloves,Coffee,Sunflower,Tomatoes,Onions,Other_1,Other_2,Other_3,No_crop
22552580,5.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,...,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0
22487045,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0
22159366,0.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0
22790149,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,...,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0
22790150,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22757331,2.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0
22552539,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0
22200293,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0
22102008,2.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0


Here, the first column, `A2` is the answers to question `A2` and the rest of the dataframe is the answers to questions about crops. 

## Country-specific Decoded objects ##

If you want to work with data from a specific country, you can make the notation even simpler by using the `Country_Decoded` subclass of `CGAP_Decoded` that's specific to a country:

In [13]:
bgd = Country_Decoded('bgd',Data)
cdi = Country_Decoded('cdi',Data)
moz = Country_Decoded('moz',Data)
nga = Country_Decoded('nga',Data)
tan = Country_Decoded('tan',Data)
uga = Country_Decoded('uga',Data)

Now you don't have to specify the country as a string:

In [14]:
print(Data.moz_A5.df.Rice.value_counts())

print(moz.A5.df.Rice.value_counts())

print(moz.col('A5','Rice').value_counts())

pd.concat([moz.col('A5','Rice'), moz.col('H28')],axis=1)

2.0    1712
1.0     619
Name: Rice, dtype: int64
2.0    1712
1.0     619
Name: Rice, dtype: int64
2.0    1712
1.0     619
Name: Rice, dtype: int64


,Rice,H28
21948170,2.0,1.0
21948180,2.0,3.0
21948779,2.0,2.0
21951744,2.0,2.0
21955646,2.0,2.0
...,...,...
23315711,2.0,NaN
23324438,2.0,2.0
23324440,1.0,NaN
23324645,NaN,NaN


## Manipulating data ##

This section introduces three methods of `CGAP_Decoded objects`.  The workhorse is `col`, which gets a column of data given a specification, if possible.  `col` takes a country as an argument, but sometimes you will want to get a column of data for one variable for some or all countries; for this, use `col_from_countries`.  To get columns of data for several variables over some or all countries, use `cols_from_countries`.  And be sure to read the warning at the end of this section!

### col ###

`col` is a `CGAP_Decoded` method that returns a pandas Series. It is there only for convenience, as it's easy to forget how to access columns, particularly those for single-answer questions. 

`Data.col('moz','A5','Rice')` is equivalent to `Data.moz_A5.df.Rice` 

`Data.col('moz','H28')` is equivalent to `Data.moz_H28.df.H28`

In [15]:
# single-answer question:
print (Data.col('moz','H28'))
print()

21948170    1.0
21948180    3.0
21948779    2.0
21951744    2.0
21955646    2.0
           ... 
23315711    NaN
23324438    2.0
23324440    NaN
23324645    NaN
23324649    3.0
Name: H28, Length: 2574, dtype: float64



In [16]:
# multi-answer question:
print (Data.col('moz','A5','Rice'))
print()

22552580    2.0
22487045    2.0
22159366    2.0
22790149    2.0
22790150    2.0
           ... 
22757331    2.0
22552539    2.0
22200293    2.0
22102008    2.0
22167545    2.0
Name: Rice, Length: 2462, dtype: float64



`col` will provide diagnostic messages and will return `None` when it can't find what you ask for:

In [17]:
# multi-answer question but the user doesn't know it
print(Data.col('moz','A5'))


moz_A5 is a multi-answer question; please specify an answer column.

Options are ['Maize', 'Beans', 'Sweet_potato', 'Sorghum', 'Rice', 'Groundnuts', 'Cowpea', 'Millet', 'Cassava', 'Potato', 'Pigeon_pea', 'Banana', 'Coconut', 'Cotton', 'Sesame', 'Mango', 'Cashew', 'Sugarcane', 'Tobacco', 'Tea', 'Avocado', 'Cocoa', 'Sisal', 'Cloves', 'Coffee', 'Sunflower', 'Tomatoes', 'Onions', 'Other_1', 'Other_2', 'Other_3', 'No_crop']
None


In [18]:
print(Data.col('moz','A6','Wheat'))

A6 is a single-answer question; ignoring Wheat
22552580     8.0
22487045     3.0
22159366    55.0
22790149     3.0
22790150     3.0
            ... 
22757331     7.0
22552539     8.0
22200293     3.0
22102008     8.0
22167545     3.0
Name: A6, Length: 2462, dtype: float64


In [19]:
# moz doesn't list wheat as an answer to question A5
Data.col('moz','A5','Wheat') is None

moz_A5.Wheat does not exist


True

In [20]:
Data.col('moz','FooBarHaHaHa!') is None

moz_FooBarHaHaHa! does not exist


True

### col_from_countries ###

If you need to get the value of a variable for all countries, use `col_from_countries`.  Here are the values of variable `A6` for Bangladesh and Cote d'Ivoire:

In [21]:
Data.col_from_countries('A6',countries=['bgd','cdi'])

,A6
1,1.0
2,1.0
3,NaN
4,1.0
5,1.0
...,...
31186932,41.0
31186933,41.0
31186934,41.0
31285239,43.0


That's pretty straightforward, but `A6` is a single-answer question.  What if you want the values over countries for a multi-answer question such as `A5`, which asks which crops are grown? Use a tuple of the question label and the name of the column that holds a particular answer:

In [22]:
Data.col_from_countries(('A5','Rice'),countries=['bgd','cdi'])

,Rice
1,1.0
2,1.0
3,NaN
4,1.0
5,1.0
...,...
31186932,1.0
31186933,1.0
31186934,1.0
31285239,1.0


### cols_from_countries ###

That's fine for single variables, but what if you want a dataframe of one or more variables over several countries? The method `cols_from_countries` builds a dataframe from several variables that can represent single-answer or multiple-answer questions. For example, here are three variables -- number of hectares (A2), whether rice is grown (A5,Rice) and whether maize is grown (A5,Maize) and country for Tanzania and Uganda:

In [23]:
Data.cols_from_countries('A2',('A5','Rice'),('A5','Maize'),'COUNTRY', countries = ['tan','uga'])
    

,A2,Rice,Maize,COUNTRY
22678091,NaN,2.0,1.0,uga
22678446,0.809717,2.0,1.0,uga
22678802,0.404858,2.0,2.0,uga
22678927,NaN,NaN,NaN,uga
22679052,3.000000,2.0,1.0,uga
...,...,...,...,...
29633312,1.214575,2.0,2.0,tan
29649585,0.607287,2.0,2.0,tan
29649586,0.607287,2.0,2.0,tan
29680313,1.214575,1.0,1.0,tan


### A warning about col_from_countries and cols_from_countries ###

You might recall that `col` tries to give you what you want, but when it can't, it prints a warning and returns `None`.  `col_from_countries` and `cols_from_countries` don't fail when they get `None` from `col`. Instead, they just don't include the countries you think you're getting:

In [24]:
wheat = Data.col_from_countries(('A5','Wheat'), countries = ['bgd','cdi','moz','nga','tan','moz'])

print(f"The dataframe includes {len(wheat)} records")

cdi_A5.Wheat does not exist
moz_A5.Wheat does not exist
tan_A5.Wheat does not exist
moz_A5.Wheat does not exist
The dataframe includes 5994 records


Instead of records from all the countries, you're getting only the 5994 records from the countries that grow wheat. You can think of this as a bug or a feature! A more troubling case arises when you ask for more than one variable:

In [25]:
wheat_and_rice = Data.cols_from_countries(('A5','Wheat'),('A5','Rice'), countries = ['bgd','cdi','moz','nga','tan','moz'])

print(f"The dataframe includes {len(wheat_and_rice)} records")
print(f"The Rice column includes {np.sum(np.isnan(wheat_and_rice['Rice']))} NaNs")
print(f"The Wheat column includes {np.sum(np.isnan(wheat_and_rice['Wheat']))} NaNs")

cdi_A5.Wheat does not exist
moz_A5.Wheat does not exist
tan_A5.Wheat does not exist
moz_A5.Wheat does not exist
The dataframe includes 16732 records
The Rice column includes 1026 NaNs
The Wheat column includes 11306 NaNs


What's happened here is that wheat isn't grown in four countries, so the records for households in those countries have NaNs for wheat.  It's the right thing to do, but be aware that it's happening!  

## Other data manipulation ##

### Adding new data objects, temporarily ###

You can add new data objects to a CGAP_Decoded object with the `add_col` method.  Here's an example of adding a new variable derived from monthly income (`D21_LZ`) and monthly outgoings (`D19_LZ`):

In [26]:
for country in countries :
    diff = Data.col(country,'D21_LZ') - Data.col(country,'D19_LZ')
    Data.add_col (country = country, 
                  label = 'INCOME_DIFF', 
                  df = diff,
                  text = "Monthly income minus monthly outgoing",
                  qtype = 'single'
                 )

`text` and `answers` default to `None` and `qtype` defaults to `'single'`, so you can get away with specifying just the positional arguments `country`, `label` and `df`, as in:

In [27]:
# make a column of random numbers that's the same length as moz
randoms = np.random.random(len(Data.col('moz','H28')))

# positional arguments are country, label and df
Data.add_col ('moz', 'random_numbers', randoms)

print(Data.__dict__.get('moz_random_numbers'))

namespace(answer=None, country='moz', df=      random_numbers
0           0.077226
1           0.482234
2           0.584238
3           0.827122
4           0.573344
...              ...
2569        0.477449
2570        0.995842
2571        0.901037
2572        0.854165
2573        0.060126

[2574 rows x 1 columns], label='random_numbers', qtype='single', text=None)


Note that `add_col` is permissive about what kind of object is passed as `df`:  Anything that can be turned into a pandas dataframe, such as the numpy array `randoms` in the previous example, will work.

#### NOTE #### 

`add_col` adds a column temporarily to a CGAP_Decoded object such as `Data`.  At present there's no way to write out the addition permanently.  That's because I wrote the encoder in a way that's too specific to CGAP data.  It's a priority to fix this so that people can save derived variables for CGAP and Manobi data objects.

### Other odds and ends (unfinished) ###

Series can be concatenated, provided they have the same indexes:

In [28]:
df = pd.concat([
    Data.col('moz','A5','Rice'),
    Data.col('moz','H28'), # "col" version
    Data.moz_H28.df.H28,  # non="col" version: you have to say H28 twice
        ], axis=1)
df

,Rice,H28,H28
21948170,2.0,1.0,1.0
21948180,2.0,3.0,3.0
21948779,2.0,2.0,2.0
21951744,2.0,2.0,2.0
21955646,2.0,2.0,2.0
...,...,...,...
23315711,2.0,NaN,NaN
23324438,2.0,2.0,2.0
23324440,1.0,NaN,NaN
23324645,NaN,NaN,NaN


Series can also be concatenated with dataframes:

In [29]:
df = pd.concat([
    Data.col('moz','A5','Rice'),
    Data.moz_A61.df
        ], axis=1)
df

,Rice,weather,pests_disease,accident,market_prices,input_prices,contract_broken,downturn_no_sale,equipment_breakdown,health
21948170,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21948180,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
21948779,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
21951744,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
21955646,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
23315711,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0
23324438,2.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0
23324440,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23324645,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Some illustrative data analysis (unfinished) ##



First let's look at the joint distribution of rice and other crops in Cote d'Ivoire:

In [30]:
x = pd.crosstab(Data.col('cdi','A5','Rice'),Data.col('cdi','A5','Groundnuts'),margins=True)
x

Groundnuts,1.0,2.0,All
Rice,,,
1.0,562,870,1432
2.0,311,1169,1480
All,873,2039,2912


We can do the same for multiple countries:

In [31]:
df = Data.cols_from_countries(
    ('A5','Rice'),('A5','Maize'),
    countries = ['cdi','moz','nga']
)

pd.crosstab(df.Rice,df.Maize)

Maize,1.0,2.0
Rice,,
1.0,2088,612
2.0,3625,1655


This tells us -- among other things -- that growing rice and maize are not independent. For example, if you grow rice in cdi, moz or nga, then the conditional probability of _not_ growing maize is 612/(612+2088) = .23, whereas if you _don't_ grow rice then you probably _will_ grow maize: the conditional probability of growing maize given that you don't grow rice is 3625/(3625+1655)= .69.

Now for something a bit more challenging:  How do respondents generate income?  Question H6 is a single-answer question that asks about a respondent's primary job, while H2B asks about sources of income:

In [32]:
print(Data.bgd_H6.text)
print(Data.bgd_H6.answers)
print()
print(Data.bgd_H2B.text)
print(Data.bgd_H2B.answers)


What is your primary job (i.e., the job where you spend most of your time)?
{'farmer': 1, 'professional': 2, 'shop_owner': 3, 'business_owner': 4, 'laborer': 5, 'other': 6}

Which of these has been your main source of income in the last year?
{'regular_job': 1, 'occasional_job': 2, 'retail_business': 3, 'services_business': 4, 'grant_pension': 5, 'family_friends': 6, 'growing_crops': 7, 'rearing_livestock': 8, 'other': 9}


In [33]:
pd.crosstab(Data.col('bgd','H6'),Data.col('bgd','H2B'), normalize='index')

H2B,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
H6,,,,,,,,,
1,0.020743,0.029908,0.036179,0.008201,0.004342,0.036662,0.721659,0.115774,0.026532
2,0.691176,0.044118,0.014706,0.073529,0.000000,0.014706,0.117647,0.014706,0.029412
3,0.023810,0.000000,0.500000,0.190476,0.023810,0.071429,0.071429,0.119048,0.000000
4,0.055944,0.017483,0.527972,0.139860,0.000000,0.017483,0.097902,0.132867,0.010490
5,0.208211,0.246334,0.017595,0.017595,0.000000,0.026393,0.164223,0.067449,0.252199
6,0.297735,0.048544,0.009709,0.045307,0.038835,0.158576,0.087379,0.135922,0.177994


So, in Bangladesh, among farmers (H6 == 1) the main sources of income are growing crops (71.6% of farmers) or raising livestock (11.4% of farmers).  Laborers (H6 == 5) have the most variable sources of income, with 20% saying they get income from a regular job (H2B ==1), 24% citing occasional work, 16% citing agriculture and 25% citing 'other' as a source of income. 

Fewer farmers in Mozambique (H6 == 1) get their income primarily from agriculture (only 45% do). Sixteen percent say they get their income primarily from occasional work (H2B == 2).

In [34]:
pd.crosstab(Data.col('moz','H6'),Data.col('moz','H2B'), normalize='index')

H2B,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
H6,,,,,,,,,
1,0.037545,0.186532,0.030989,0.035161,0.041120,0.087008,0.510131,0.051847,0.019666
2,0.726415,0.094340,0.018868,0.028302,0.018868,0.009434,0.103774,0.000000,0.000000
3,0.176471,0.117647,0.294118,0.058824,0.000000,0.117647,0.117647,0.117647,0.000000
4,0.083333,0.116667,0.233333,0.305556,0.005556,0.038889,0.172222,0.022222,0.022222
5,0.607477,0.242991,0.009346,0.037383,0.000000,0.009346,0.065421,0.028037,0.000000
6,0.281437,0.299401,0.011976,0.071856,0.041916,0.065868,0.089820,0.035928,0.101796


The raw frequencies are instructive: Among 1870 farming households (76% of those surveyed in moz), 303 say their primary source of income is occasional work and 144 say it is family or friends (H2B == 7).  In fact, 49.7% of the farming households say their primary source of income is not either agriculture (H2B == 7) or raising livestock (H2B==8).  (Exercise for the reader: Contrast Mozambique with Nigeria.)

In [35]:
pd.crosstab(Data.col('moz','H6'),Data.col('moz','H2B'))

H2B,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
H6,,,,,,,,,
1,63,313,52,59,69,146,856,87,33
2,77,10,2,3,2,1,11,0,0
3,3,2,5,1,0,2,2,2,0
4,15,21,42,55,1,7,31,4,4
5,65,26,1,4,0,1,7,3,0
6,47,50,2,12,7,11,15,6,17


### Building a classifier with sklearn and data objects ###

The `cols_from_countries` method of CGAP_Decoded objects makes it easy to assemble dataframes for multiple variables for multiple countries. To illustrate, the following class defines a classifier for one or more countries: 


In [36]:
import operator
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score


class Country_Classifier ():
    def __init__(self,cgap_decoded_obj, countries, X, y, model_class, report = False):
        self.Data = cgap_decoded_obj
        self.X,  self.y = self.Xy_data(countries, X, y, report)
        self.model_class = model_class
        
    def Xy_data (self,countries,X,y, report):
        # make one table of X and y so indices remain aligned 
        # when we drop rows that contain NaNs
        Xy = self.Data.cols_from_countries(*X, y, countries= countries)
        n0 = len(Xy)
        
        # drop rows that contain NaNs
        Xy.dropna(axis=0,inplace=True)
        n1 = len(Xy)
        
        if report : print(f"{countries}:  Removed {n0-n1} rows, loss = {(n0-n1)/n0:.4f}\n")
        
        # reset the index of Xy
        Xy = Xy.reset_index(drop=True)
        
        # return X and y as separate dataframes. If y denotes a column for a 
        # multi-answer then it will be a tuple and the column to drop is the 
        # second value in the tuple
        
        if type(y) in [tuple,list]:
            return Xy.drop(y[1],axis=1) , Xy[y[1]]
        else:
            return Xy.drop(y,axis=1) , Xy[y]
    
    def train_model (self):
        self.model = self.model_class.fit(self.X,self.y) 
        
    def score (self, X = None, y = None):
        """ Returns the model score when tested with X and y.  If these are None
        then the default is X = self.X, y = self.y. However, X and y can be new
        data (e.g., from out of sample) """
        X0 = self.X if X is None else X
        y0 = self.y if y is None else y
        return self.model.score(X0,y0)
    
    def cv_score (self, X = None, y = None, k = 5):
        """ Returns the k-fold cross_validated model score when tested with 
        X and y; also returns the scores themselves.  If X and y are None then 
        the default is X = self.X, y = self.y. """
        X0 = self.X if X is None else X
        y0 = self.y if y is None else y
        cv_scores = cross_val_score(self.model, X0, y0, cv=k)
        return np.mean(cv_scores), cv_scores
    
    def majority_class (self, y = None):
        """ Returns the majority class probability for y and the majority class 
        label.  If y is None then the default is y = self.y. """
        y0 = self.y if y is None else y
        vc = dict(y0.value_counts())
        max_key, max_count = max(vc.items(), key = operator.itemgetter(1))
        return max_count/sum(vc.values()), max_key
    
    
        
ICP = Country_Classifier(
    cgap_decoded_obj = Data,
    countries = ['cdi','nga','tan'],
    X = ['A38',('A41','my_legacy'),'NUM_KIDS'],
    y = ('A41','want_children_continue'),
    model_class = RandomForestClassifier(max_features=None),
    report=True
    )


ICP.train_model()
print(f"Model score on training data: {ICP.score()}")
print(f"Cross-validated model score: {ICP.cv_score()}")
print(f"Majority class in training data: {ICP.majority_class()}")


['cdi', 'nga', 'tan']:  Removed 1917 rows, loss = 0.2121

Model score on training data: 0.7668866732200533
Cross-validated model score: (0.763516361127538, array([0.76350877, 0.74227528, 0.70224719, 0.78160112, 0.82794944]))
Majority class in training data: (0.6501895801151524, 1.0)


Here's an analysis that compares predictions in one country given X from that country and a model from another country. 

In [37]:
def intercountry_predictions(cgap_decoded_obj, X,y,model_class, countries):
    
    results = pd.DataFrame(columns=['mc','self','self-mc','n']+countries, index=countries)
    diffs = pd.DataFrame(columns=countries, index=countries)
    country_classifiers = {}
    
    # First train models for each country on data for that country
    
    for country in countries:
        CC = Country_Classifier(
            cgap_decoded_obj = cgap_decoded_obj,
            countries = [country], 
            X = X, y = y, model_class = model_class()
        )
        CC.train_model()      
        country_classifiers[country] = CC
        
        # within-sample classifier score
        results.loc[country,'self'] = round(CC.score(),3)
        
        # majority class prediction score
        results.loc[country,'mc'] = round(CC.majority_class()[0],3)
        
        # improvement of classifier over majority class
        results.loc[country,'self-mc']=round(CC.score() - CC.majority_class()[0],3)
        
        # number of non-NaN records for available to this classifier
        results.loc[country,'n'] = len(CC.y)
        
    
    # Now use these models to predict y's between countries
    for country1 in countries:
        CC1 = country_classifiers[country1]
        
        for country2 in countries:       
            if country1 == country2:
                results.loc[country1,country2] = round(CC1.score(),3)
                diffs.loc[country1,country2] = 0
                  
            else:
                # Use country1's classifier to predict other countries
                CC2 = country_classifiers[country2]
                
                # score of country1's  model on c2 data: how well the model predicts c2
                c12 =  CC1.score(CC2.X,CC2.y) 
                
                # how much worse it is to use c1 model to predict c2 than it is to use c2 model
                c12_loss = c12 - CC2.score()
        
                results.loc[country1,country2] = round(c12,3)
                diffs.loc[country1,country2] = round(c12_loss,3)
            
    return results, diffs
 
X = ['A38',('A41','my_legacy'),'NUM_KIDS']
y = ('A41','want_children_continue')

results, diffs = intercountry_predictions(Data, X, y, RandomForestClassifier, countries)
 
print ("Accuracy when row country predicts column country\n")
print(results)
print()
print ("Loss of accuracy in column country predictions when row country predicts column country\n")
print(diffs)

Accuracy when row country predicts column country

        mc   self self-mc     n    bgd    cdi    moz    nga    tan    uga
bgd  0.693  0.695   0.002  2371  0.695   0.46  0.366  0.421  0.301  0.373
cdi  0.541  0.639   0.098  2281  0.542  0.639  0.777  0.817  0.795  0.737
moz  0.645  0.797   0.152  1316  0.526  0.611  0.797  0.812  0.806  0.758
nga  0.657  0.849   0.192  2232  0.539  0.628  0.786  0.849  0.816  0.771
tan   0.74  0.819   0.079  2608  0.538   0.62  0.785  0.841  0.819   0.77
uga  0.679  0.777   0.098  2071  0.536  0.617  0.786  0.832  0.811  0.777

Loss of accuracy in column country predictions when row country predicts column country

       bgd    cdi    moz    nga    tan    uga
bgd      0 -0.179 -0.431 -0.428 -0.517 -0.404
cdi -0.153      0 -0.021 -0.032 -0.023  -0.04
moz  -0.17 -0.028      0 -0.037 -0.012 -0.019
nga -0.156 -0.011 -0.011      0 -0.003 -0.006
tan -0.158 -0.019 -0.012 -0.008      0 -0.007
uga -0.159 -0.022 -0.011 -0.017 -0.008      0
